# Trajectory 사람 이동 예측
trajectory 데이터에서 총 180명 정도의 데이터가 존재 <br>
개인 별로 어떻게 움직일지를 lstm, gru (파라미터가 좀 작을 때 사용하는 모델),transfomer 모형을 활용해서 학습해볼 예정 <br>
## 표본 선정 기준<br>
1. 수집 일 수가 많을 것 <br>
2. 일정하게 데이터가 존재할 것 <br>
결과적으로 총 6명을 학습 할 예정이다. <br>

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd drive

/content/drive


In [3]:
cd MyDrive

/content/drive/MyDrive


In [4]:
cd DeepLearning_14Team/

/content/drive/MyDrive/DeepLearning_14Team


In [ ]:
import os
import pandas as pd
list_ = os.listdir('sorted_data/128/Trajectory/')
sorted_41 = []
for i in range(len(list_)):
    file_ = open('sorted_data/128/Trajectory/' + list_[i], 'r') 
    file = file_.readlines()
    file = file[6:]
    for j in range(len(file)):
        tmp = file[j].split(',')
        sorted_41.append([float(tmp[0]), float(tmp[1]), tmp[-2] + ' ' +tmp[-1][:-1]])

df = pd.DataFrame(sorted_41)
df[2] = pd.to_datetime(df[2])
df.sort_values(by = [2],inplace=True)
df = df.reset_index()
df = df[[0,1,2]]

KeyboardInterrupt: ignored

In [44]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving sorted_128_last.csv to sorted_128_last (2).csv
User uploaded file "sorted_128_last.csv" with length 58497629 bytes


In [ ]:
len(df)

1208500

In [46]:
df = pd.read_csv('sorted_128_last (2).csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [1]:
import pandas as pd
df = pd.read_csv('sorted_128_last_1000.csv')

In [6]:
df

,Unnamed: 0,Unnamed: 0.1,0,1,2,3,4
0,0,0,40.075850,116.327817,2007-04-14,00:56:28,"Unnamed Road, Changping Qu, Beijing Shi, 중국"
1,1,1,40.075833,116.327583,2007-04-14,00:56:46,0
2,2,2,40.076000,116.327567,2007-04-14,00:59:24,0
3,3,3,40.076300,116.327500,2007-04-14,00:59:50,0
4,4,4,40.076600,116.327417,2007-04-14,01:00:15,0
...,...,...,...,...,...,...,...
995,995,995,40.255033,116.299217,2007-04-15,04:50:17,0
996,996,996,40.255067,116.299600,2007-04-15,04:50:43,0
997,997,997,40.255217,116.299767,2007-04-15,04:51:01,중국 베이징 시 베이징시
998,998,998,40.255483,116.299717,2007-04-15,04:51:21,0


# 데이터 분리하기
1. 데이터가 너무 많다는 점
2. 어느 곳에 방문할지에 대한 정보가 좀 더 중요하다는 점
3. 사람이 걸을 수 있는 정도가 한정되어 있다는 점 <br>
세가지를 고려할 때 모두를 학습시키기 보다는 일부분을 쪼개서 학습시키는 것이 합리적

In [ ]:
a = []
for i in range(1208499):
    if i % 30 == 0:
        diff = df[2].iloc[i + 1] - df[2].iloc[i]
        a.append(diff)

In [ ]:
result = a[0]
for i in range(1,len(a)):
    result += a[i]

In [ ]:
result / len(a)

Timedelta('0 days 00:01:27.645094826')

# 30분 단위로 쪼개서 이 사람의 미래 예측하는 형태
    평균이 1분 27초 정도되니 30정도 곱해서 30분 단위로 예측해볼 것임

In [2]:
df_diff_30 = []
for i in range(0,len(df),30):
    df_diff_30.append(df.iloc[i])

In [ ]:
len(df_diff_30)

40284

In [3]:
df_diff_30_pd = pd.DataFrame(df_diff_30)

In [4]:
df_diff_30_pd

,Unnamed: 0,Unnamed: 0.1,0,1,2,3,4
0,0,0,40.075850,116.327817,2007-04-14,00:56:28,"Unnamed Road, Changping Qu, Beijing Shi, 중국"
30,30,30,40.076483,116.321833,2007-04-14,01:10:40,0
60,60,60,40.051017,116.323883,2007-04-14,01:35:35,"중국 Beijing Shi, Haidian Qu, 安宁庄东路北口"
90,90,90,40.046900,116.320683,2007-04-14,01:48:46,0
120,120,120,40.040267,116.325833,2007-04-14,01:59:23,"중국 Beijing Shi, Haidian Qu, 安宁庄前街"
150,150,150,40.036267,116.310317,2007-04-14,02:18:03,0
180,180,180,40.014733,116.311717,2007-04-14,02:33:58,"47 Zhongguancun N St, Haidian Qu, Beijing Shi, 중국"
210,210,210,39.991983,116.309617,2007-04-14,02:53:37,0
240,240,240,39.985300,116.309717,2007-04-14,03:16:52,"중국 Beijing Shi, Haidian Qu, Zhong Guan Cun, 海淀桥东"
270,270,270,39.979617,116.313667,2007-04-14,03:27:36,0


In [ ]:
df_diff_30_pd[0], df_diff_30_pd[1], df_diff_30_pd[2]

KeyError: ignored

# 30분 단위로 쪼갠다!

In [ ]:
df_128_30[0]

KeyError: ignored

In [ ]:
df_128_30 = pd.DataFrame(df_diff_30)

In [ ]:
x, y = df_128_30['0'], df_128_30['1']

In [47]:
x, y = df['0'], df['1']

In [49]:
y

0          116.327817
1          116.327583
2          116.327567
3          116.327500
4          116.327417
              ...    
1110818    116.410697
1110819    116.410728
1110820    116.410753
1110821    116.410745
1110822    116.410732
Name: 1, Length: 1110823, dtype: float64

# 모델 학습 데이터 준비
    x, y 차원을 맞추는 것이 어려운 점이었음
    lstm 모델 2개를 활용해서 한 차원 당 1개의 모델을 사용하도록 하였음.
    

In [50]:
import random
import numpy as np
import torch

# multivariate data preparation
from numpy import array
from numpy import hstack
 
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)-n_steps):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-2], sequences[end_ix, -2:]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)
 
# x, y
# define input sequence
in_seq1 = x.values
in_seq2 = y.values
#out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
#out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, in_seq1, in_seq2))

In [51]:
df = split_sequences(dataset, 5)

In [52]:
df

(array([[[ 40.07585   , 116.32781667],
         [ 40.07583333, 116.32758333],
         [ 40.076     , 116.32756667],
         [ 40.0763    , 116.3275    ],
         [ 40.0766    , 116.32741667]],
 
        [[ 40.07583333, 116.32758333],
         [ 40.076     , 116.32756667],
         [ 40.0763    , 116.3275    ],
         [ 40.0766    , 116.32741667],
         [ 40.07686667, 116.32736667]],
 
        [[ 40.076     , 116.32756667],
         [ 40.0763    , 116.3275    ],
         [ 40.0766    , 116.32741667],
         [ 40.07686667, 116.32736667],
         [ 40.07706667, 116.32733333]],
 
        ...,
 
        [[ 39.9358249 , 116.4107033 ],
         [ 39.9357566 , 116.4106783 ],
         [ 39.9357349 , 116.4106799 ],
         [ 39.9357349 , 116.4106966 ],
         [ 39.9357683 , 116.4107283 ]],
 
        [[ 39.9357566 , 116.4106783 ],
         [ 39.9357349 , 116.4106799 ],
         [ 39.9357349 , 116.4106966 ],
         [ 39.9357683 , 116.4107283 ],
         [ 39.9358216 , 116.4107533 ]

# GRU 모델 학습

In [53]:
cuda0 = torch.device('cuda:0')

In [11]:
import torch

In [54]:
class MV_GRU(torch.nn.Module):
    def __init__(self,n_features,seq_length):
        super(MV_GRU, self).__init__()
        self.n_features = n_features
        self.seq_len = seq_length
        self.n_hidden = 100 # number of hidden states
        self.n_layers = 300 # number of LSTM layers (stacked)
    
        self.l_lstm = torch.nn.GRU(input_size = n_features, 
                                 hidden_size = self.n_hidden,
                                 num_layers = self.n_layers, 
                                batch_first = True,
                                dropout=0.1).to(device=cuda0)
        # according to pytorch docs LSTM output is 
        # (batch_size,seq_len, num_directions * hidden_size)
        # when considering batch_first = True
        self.l_linear = torch.nn.Linear(self.n_hidden*self.seq_len, 1).to(device=cuda0)
        self.relu = torch.nn.ReLU()
    
    def init_hidden(self, batch_size):
        # even with batch_first = True this remains same as docs
        hidden_state = torch.zeros(self.n_layers,batch_size, self.n_hidden, device = cuda0)
#        cell_state = torch.zeros(self.n_layers,batch_size, self.n_hidden, device = cuda0)
        self.hidden = (hidden_state)
    
    
    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        
        lstm_out, self.hidden = self.l_lstm(x, self.hidden)
        lstm_out = self.relu(lstm_out)
        # lstm_out(with batch_first = True) is 
        # (batch_size,seq_len,num_directions * hidden_size)
        # for following linear layer we want to keep batch_size dimension and merge rest       
        # .contiguous() -> solves tensor compatibility error
        x = lstm_out.contiguous().view(batch_size,-1)
        return self.l_linear(x)

In [21]:
import torch
class GRUNet(torch.nn.Module):
    def __init__(self, input_dim = 5, hidden_dim = 100, output_dim = 1, n_layers = 50, drop_prob=0.2):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = torch.nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = torch.nn.Linear(hidden_dim, output_dim)
        self.relu = torch.nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device= cuda0)
        return hidden

In [37]:
mv_net_test = MV_GRU(n_features, n_timesteps)

In [ ]:
mv_net_test.n_layers
mv_net_test.n_hidden

In [55]:
n_features = 1 # this is number of parallel inputs
n_timesteps = 5 # this is number of timesteps

# convert dataset into input/output
X, y = split_sequences(dataset, n_timesteps)
print(X.shape, y.shape)

# create NN
mv_net_x = MV_GRU(n_features, n_timesteps)
mv_net_y = MV_GRU(n_features, n_timesteps)
#criterion = torch.nn.MSELoss().to(device=cuda0)
criterion = torch.nn.L1Loss().to(device=cuda0)
 # reduction='sum' created huge loss value
optimizer_x = torch.optim.Adam(mv_net_x.parameters(), lr=1e-1)
optimizer_y = torch.optim.Adam(mv_net_y.parameters(), lr=1e-1)

# MV_GRU(n_features, n_timesteps) # n_hidden, n_layers
mv_net_test = MV_GRU(n_features, n_timesteps)
layer = mv_net_test.n_layers
hidden = mv_net_test.n_hidden

train_episodes = 30
batch_size = 50

(1110818, 5, 2) (1110818, 2)


In [ ]:
type(X) 
# 여기서 타입 에러가 발생했음 # float 타입으로 결과를 만들어야 했는데
# 위에서 string 타입으로 데이터 프레임을 만들어내서 문제가 발생

numpy.ndarray

In [ ]:
X

array([[[ 40.07585   , 116.32781667],
        [ 40.07648333, 116.32183333],
        [ 40.05101667, 116.32388333],
        [ 40.0469    , 116.32068333],
        [ 40.04026667, 116.32583333]],

       [[ 40.07648333, 116.32183333],
        [ 40.05101667, 116.32388333],
        [ 40.0469    , 116.32068333],
        [ 40.04026667, 116.32583333],
        [ 40.03626667, 116.31031667]],

       [[ 40.05101667, 116.32388333],
        [ 40.0469    , 116.32068333],
        [ 40.04026667, 116.32583333],
        [ 40.03626667, 116.31031667],
        [ 40.01473333, 116.31171667]],

       ...,

       [[ 39.9705499 , 116.4605233 ],
        [ 39.9647216 , 116.464055  ],
        [ 39.98799   , 116.4282633 ],
        [ 39.9872383 , 116.4025066 ],
        [ 39.986165  , 116.3647599 ]],

       [[ 39.9647216 , 116.464055  ],
        [ 39.98799   , 116.4282633 ],
        [ 39.9872383 , 116.4025066 ],
        [ 39.986165  , 116.3647599 ],
        [ 39.9852083 , 116.3309099 ]],

       [[ 39.98799   , 116.

# 여기서도 어려웠던 점
    인풋 사이즈를 맞춰줘야 하는데 그 자체가 코드를 모두 이해하여야 했기에 엄청난 어려움이 있었다.
    ★시계열 데이터이기 때문에 마지막 배치의 경우에 문제가 존재했음 -> 이 부분을 처리하는 것이 중요!
    텐서 자체에서 문제가 존재했고, 데이터를 텐서로 변환하는 과정에서 소수점의 사라짐이 존재하게 되었음 이러한 문제점을 해결하는 방법은?


In [ ]:
import torch
print(torch.cuda.device_count())

1


In [16]:
cuda0 = torch.device('cuda:0')

In [ ]:
cuda0

device(type='cuda', index=0)

In [ ]:
print(torch.cuda.get_device_name(0))

Tesla P100-PCIE-16GB


In [ ]:
y.shape

NameError: ignored

In [ ]:
import torch
torch.cuda.is_available()

In [19]:
torch.set_printoptions(precision=10)

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [35]:
ls

drive/  sample_data/  sorted_128_last_1000.csv


In [36]:
cd drive/MyDrive/DeepLearning_14Team

/content/drive/MyDrive/DeepLearning_14Team


In [ ]:
mv_net_x.train()
mv_net_y.train()

for t in range(train_episodes):
    for b in range(0,len(X),batch_size):
        inpt = X[b:b+batch_size,:,:]
        target = y[b:b+batch_size]

        # x, y 트레인셋
        x_batch = torch.tensor(inpt[:,:,0],dtype=torch.float32, device = cuda0) # 텐서로 바꾸면서
        y_batch = torch.tensor(inpt[:,:,1],dtype=torch.float32, device = cuda0)
        
        if x_batch.shape == (batch_size,n_timesteps):
            x_batch = x_batch.reshape(batch_size,n_timesteps,1) # 3 채널로 맞춰줘야 함
            y_batch = y_batch.reshape(batch_size,n_timesteps,1)

            mv_net_x.init_hidden(x_batch.size(0))
            mv_net_y.init_hidden(y_batch.size(0))

            # x,y의 예측 값
            output_x = mv_net_x(x_batch)
            output_y = mv_net_y(y_batch)

            # x,y의 정답
            target_x_batch = torch.tensor(target[:, 0],dtype=torch.float32, device = cuda0)
            target_y_batch = torch.tensor(target[:, 1],dtype=torch.float32, device = cuda0)


        #    lstm_out, _ = mv_net.l_lstm(x_batch,nnet.hidden)    
        #    lstm_out.contiguous().view(x_batch.size(0),-1)

            # loss function이 두개의 합이 최소가 되는 쪽으로 loss를 구함
            loss = criterion(output_x.view(-1), target_x_batch) + criterion(output_y.view(-1), target_y_batch)
            loss.backward()

            optimizer_x.step()        
            optimizer_x.zero_grad() # x가 0으로 흐르도록 함

            optimizer_y.step()
            optimizer_y.zero_grad() # y가 0으로 흐르도록 함
    # 추가 정보

    PATH = "gru_checkpoint/GRU + MAE episode : {}, batch_size : {}, layer : {}, hidden : {}, epoch : {}, loss : {}".format(train_episodes,batch_size,layer,hidden, t, loss.items()) + ".pt"
    torch.save({
            'epoch': t,
            'model_state_dict': mv_net_x.state_dict(),
            'optimizer_state_dict': optimizer_x.state_dict(),
            'loss': loss,
            }, PATH)
    torch.save({
            'epoch': t,
            'model_state_dict': mv_net_x.state_dict(),
            'optimizer_state_dict': optimizer_y.state_dict(),
            'loss': loss,
            }, PATH)
    print('step : ' , t , 'loss : ' , loss.item())

In [ ]:
https://blog.floydhub.com/gru-with-pytorch/